In [3]:
%env KERAS_BACKEND=theano
from preprocess import MSRII
from preprocess.c3d import C3DFeatureNet
from preprocess.video import Video
from network.model import FC4Net

env: KERAS_BACKEND=theano


In [5]:
video = Video('/data-disk/MSRII/videos/1.avi')

c3dnet = C3DFeatureNet(feature_file=None)
c3dnet.load()
y = c3dnet.extract_feature(video)

fc4net = FC4Net()
fc4net.load_weights('network/FC4Net_weights.h5')
pred = fc4net.predict(y, batch_size=32)

C3D-Network is ready.


In [6]:
dataset = MSRII.Dataset('/data-disk/MSRII/')

In [9]:
import numpy as np

seg_metas = dataset.video_metas[0].seg_metas
target = np.zeros(len(video.frames))
for seg_meta in seg_metas:
    s = seg_meta['start']
    t = seg_meta['duration']
    target[s:s + t] = 1

predict = np.zeros(len(video.frames))
p = pred.reshape(len(pred))
for t in np.nditer(np.where(p == 1)):
    predict[t * 16:(t + 1) * 16] = 1

交集 = np.logical_and(target == 1, predict == 1)
聯集 = np.logical_or(target == 1, predict == 1)

print('tIOU:', sum(交集 == 1) / sum(聯集 == 1))

tIOU: 0.635379061372
